In [2]:
import numpy as np

In [19]:
from numpy import pi, cos
from numpy.linalg import cond, det
from scipy.linalg import norm, svd

n = 12
A = np.ones((n+1, n+1))
for i in range(1, n+1):
    for j in range(1, n+1):
        A[i,j] = 1/(2*i)**j
sumA = sum(sum(A))
print(f"sumA: {sumA}")

condA = cond(A, p=2)
print(f"condA: {condA}")

comment1 = 'yes' if condA <= 1 else 'no'
print(f"comment1: {comment1}")

detA = det(A)
print(f"detA: {detA}")

_, s, _ = svd(A)
SumsvA = sum(s)
print(f"SumsvA: {SumsvA}")


sumA: 27.224108678060876
condA: 3.161309735587093e+17
comment1: no
detA: -1.2411566175315346e-80
SumsvA: 7.520671865894069


In [22]:
x_pts = np.array([1, 1/2, 1/4, 1/6, 1/8, 1/10])
f1g = lambda x : (x+1) * cos(2 * pi * x)
y_pts = f1g(x_pts)

from scipy.linalg import solve
veca = solve(A[:6, :6], y_pts)
print(f"veca: {veca}")

veca: [  1.00798809   0.7310131  -16.13915809 -44.18283957 153.30979796
 -92.72680149]


In [26]:
from numpy import zeros, exp
f1 = lambda x : (x-1)**4
f2 = lambda x : 3 - exp(x)
g = lambda x : f1(x) - f2(x)
g_prime = lambda x : 4*(x-1)**3 + exp(x)
newton_step = lambda xn : xn - g(xn)/g_prime(xn)

x0 = 5/2
x1 = newton_step(x0)
x2 = newton_step(x1)
x3 = newton_step(x2)
print(f"x1: {x1}, x2: {x2}, x3: {x3}")

tol = 1e-8
xn = x0
xe = None
while True:
    xe = newton_step(xn)
    if (abs(xe - xn) < tol):
        break
    xn = xe
print(f"xe: {xe}")

f1f2 = xe
print(f"f1f2: {f1f2}")

x1: 1.9453422637809399, x2: 1.4832195389998697, x3: 1.1823751896755503
xe: 1.0985808071939192
f1f2: 1.0985808071939192


In [32]:
from numpy import ones, sqrt, pi, cos, exp, linspace
from scipy.special import erf

N = 10
h = 1/N
x = linspace(0, 1, N+1)

w = np.ones(N+1)
w[1::2] = 4
w[2:-1:2] = 2

f3 = lambda x : exp(x) * sqrt(1-x)
f = f3(x)

simpf = (w.T @ f) * h/3
print(f"simpf: {simpf}")

true_sol = 1/2 * (sqrt(pi) * erf(1) * exp(1)) - 1
errsimp = abs(simpf - true_sol)
print(f"errsimp: {errsimp}")


simpf: 1.0229712594932272
errsimp: 0.0071072097854771865


In [33]:
y = linspace(0, 1, N+1)
X,Y = np.meshgrid(x,y,indexing='ij')
j_integrand = lambda x,y : cos(2*x+y) * sqrt(x*y)
F = j_integrand(X,Y)
simp2dF = h**2/9 * (w @ F @ w.T)
print(f"simp2dF: {simp2dF}")

simp2dF: -0.09026541105629893


In [56]:
from numpy import ones, linspace, loadtxt, pi, sqrt, cos, sin
from scipy.interpolate import CubicSpline
from scipy.fft import fft
from scipy.linalg import lstsq

tdata = np.array([1,2,3,4,5,6,7,8,9])
ydata = np.array([94.88,79.48,61.95,99.67,72.34,66.05,56.64,41.51,31.08])

spl = CubicSpline(tdata, ydata, bc_type='not-a-knot')
teval = np.arange(0, 10) + 0.5
yspteval = spl(teval)
print(f"yspteval: {yspteval}")

M = np.column_stack((
    np.ones(9),
    tdata,
    tdata**2,
    sqrt(tdata),
    sin(pi * tdata/2)
))
xLS,_,_,_ = lstsq(M, ydata)
print(f"xLS: {xLS}")
M_eval = np.column_stack((
    np.ones(10),
    teval,
    teval**2,
    sqrt(teval),
    sin(pi * teval/2)
))
yLSteval = M_eval @ xLS
print(f"yLSteval: {yLSteval}")

w = fft(ydata)
m = 9
a0 = w[0]/m

b_init = np.zeros(5)
for k in range(1, 5):
    b_init[k] = -2/m * w[k].imag
b = b_init[1:]

a = np.array([a0.real, -2.3688, 9.9369, 16.6633, 3.5819])
ydfteval = [a[0] + np.sum([a[i]*cos(2*pi*i*t/m) + b_init[i]*sin(2*pi*i*t/m) for i in range(1, 5)]) for t in teval]
print(f"ydfteval: {ydfteval}")

yspteval: [63.64970753 95.07255849 63.35494151 82.43767548 90.52935657 65.97364824
 62.98605048 49.04714984 35.14785016 30.4257492 ]
xLS: [ 171.44743728   60.70423126   -3.34496404 -141.0944602     6.16713255]
yLSteval: [105.55528355  86.53371981  74.85123919  74.61205131  81.93522856
  77.60052832  60.61768555  47.81103018  38.76306718  15.73311762]
ydfteval: [103.45877763268165, 55.83837394245035, 87.25902437128326, 88.68836182411269, 66.29096666666665, 62.456762819472075, 51.172810543205635, 29.08075939088296, 59.354162809244656, 103.45877763268166]


In [83]:
from numpy.linalg import solve, norm
from numpy import diag, exp, ones, zeros, linspace, inf

def do(N=5):
    h = 1/N
    x = linspace(0, 1, N+1)
    x_pts = x[1:-1]
    A = diag([-2] * (N-1)) + diag(np.ones(N-2), 1) + diag(np.ones(N-2), -1)
    B = np.diag(exp(x_pts - 1)) @ (diag(np.ones(N-2),1) - diag(np.ones(N-2), -1))
    C = np.eye(N-1)
    R = np.zeros(N-1)
    R[0] = 1/h**2 - 1/(2*h) * exp(h-1)

    Uapprox = solve(-1/h**2 * A - 1/(2*h) * B + C, R)
    return Uapprox, B.sum()
Uapprox, sumB = do()
print(f"sumB: {sumB}")
print(f"Uapprox: {Uapprox}")


sumB: -0.36940178896076037
Uapprox: [0.71321487 0.47839554 0.28614985 0.12878524]


In [84]:
true_u = lambda x : (1 - exp(1-x))/(1 - exp(1))
true_40 = true_u(linspace(0,1,41)[1:-1])
true_80 = true_u(linspace(0,1,81)[1:-1])
U1,_ = do(40)
U2,_ = do(80)
E1 = norm(U1 - true_40, ord=inf)
E2 = norm(U2 - true_80, ord=inf)
E = [E1, E2]
print(f"E: {E}")

E: [1.2536427035914066e-06, 3.135926765485131e-07]
